In [ ]:
### Importar librerias

In [3]:
import pandas as pd
import numpy as np
from bdatos import Bdatos
from ingestar import Ingestar

import warnings
warnings.filterwarnings("ignore")

ImportError: cannot import name 'Bdatos' from 'bdatos' (/workspaces/Proyecto_integrador_V/src/proyecto_integrador_v/bdatos.py)